Виктория Карпейкина, Юлия Щучкина, Максим Бочаров

Тема проекта: "Классификация криптовалют на основе их рыночных показателей".

****ПАРСИНГ ДАННЫХ****

In [202]:
pip install python-binance

Note: you may need to restart the kernel to use updated packages.


In [232]:
# Здесь задали API-ключи
apikey = ''
secret = ''

In [204]:
# Подключились к Бинансу
import pandas as pd
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(apikey, secret)


In [205]:
# Получили все торгуемые пары
tickers = client.get_all_tickers()

In [206]:
df = pd.DataFrame(tickers)
coins = df['symbol'].values.tolist()

In [233]:
# Отобрали валютные пары типа "криптовалюта/USDT"
col = []
for item in coins:
    ans = item.endswith('USDT')
    r_1 = item.startswith('USD')
    if ans == True:
        if r_1 == False:
            col.append(item)
                
#col

In [222]:
len(col)

442

Теперь соберем котировки для первых 152 из полученных криптовалют с 1.01.2020 по 10.05.2023

In [217]:
# Записали котировки биткойна в таблицу
import time
time.sleep(3)

klines = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1DAY, "1 Jan, 2020","10 May, 2023")

    
k = pd.DataFrame(klines)

kk = pd.DataFrame()            
kk['BTCUSDT'] = k[4]
kk

,BTCUSDT
0,7200.85000000
1,6965.71000000
2,7344.96000000
3,7354.11000000
4,7358.75000000
...,...
1221,28848.20000000
1222,28430.10000000
1223,27668.79000000
1224,27628.27000000


In [218]:
# Записали котировки эфира в таблицу
time.sleep(10)

klines = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_1DAY, "1 Jan, 2020","10 May, 2023")
    
kar = pd.DataFrame(klines)

kar[4]

0        130.77000000
1        127.19000000
2        134.35000000
3        134.20000000
4        135.37000000
            ...      
1221    1896.33000000
1222    1870.40000000
1223    1847.56000000
1224    1846.52000000
1225    1840.84000000
Name: 4, Length: 1226, dtype: object

In [219]:
# Сделали таблицу для всех котировок
df_price = pd.DataFrame()
df_price = kk
df_price['ETHUSDT'] = kar[4]
df_price

,BTCUSDT,ETHUSDT
0,7200.85000000,130.77000000
1,6965.71000000,127.19000000
2,7344.96000000,134.35000000
3,7354.11000000,134.20000000
4,7358.75000000,135.37000000
...,...,...
1221,28848.20000000,1896.33000000
1222,28430.10000000,1870.40000000
1223,27668.79000000,1847.56000000
1224,27628.27000000,1846.52000000


На примере биткойна и эфира мы получили котировки за период в 1226 дней. Теперь сделаем функцию для этого процесса

In [220]:
# Функция для получения котировок конкретной валютной пары
def price(coin):
    time.sleep(5)

    klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1DAY, "1 Jan, 2020", "10 May, 2023")
    kar = pd.DataFrame(klines)
    if kar.empty == False:
        ans = kar[4]
        return ans
    else:
        return kar

Поскольку через API Binance можно сделать только ограниченное количество запросов в минуту, час, сутки, испозьуем метод time.sleep() и для удобства разобьем запросы на последовательные группы по 15 тикеров.

In [221]:
# Начинаем с col[2], т.к. биткойн (№0) и эфир (№1) мы уже вставили в таблицу
for item in col[2:15]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,XLMUSDT,ONTUSDT,TRXUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,0.04529000,0.52750000,0.01322000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,0.04361000,0.50370000,0.01284000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,0.04566000,0.52780000,0.01344000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,0.04565000,0.53160000,0.01344000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,0.04544000,0.54830000,0.01353000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,0.09240000,0.22480000,0.07025000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,0.09180000,0.22260000,0.06920000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,0.08940000,0.20470000,0.06850000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,0.08900000,0.20680000,0.06906000


In [223]:
for item in col[15:30]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,ETCUSDT,ICXUSDT,NULSUSDT,VETUSDT,LINKUSDT,WAVESUSDT,ONGUSDT,HOTUSDT,ZILUSDT,ZRXUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,4.50350000,0.11460000,0.23080000,0.00535800,1.81110000,1.04460000,0.09560000,0.00062750,0.00454000,0.18250000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,4.22830000,0.10960000,0.22070000,0.00509400,1.73660000,0.94590000,0.09500000,0.00060350,0.00438000,0.17520000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,4.57680000,0.11420000,0.22840000,0.00532700,1.80400000,0.95260000,0.09950000,0.00067940,0.00454000,0.18210000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,4.75980000,0.11390000,0.23370000,0.00562000,1.82420000,0.94580000,0.09810000,0.00067960,0.00461000,0.18640000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,4.87840000,0.11350000,0.23400000,0.00546500,1.79570000,0.88720000,0.09980000,0.00067080,0.00463000,0.18610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,18.94000000,0.27310000,0.23290000,0.02024000,6.92000000,1.80000000,0.27070000,0.00178300,0.02639000,0.24060000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,18.73000000,0.26680000,0.22930000,0.02009000,6.91100000,1.80600000,0.26620000,0.00175500,0.02608000,0.23730000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,18.27000000,0.24200000,0.21330000,0.01930000,6.58200000,1.60400000,0.24830000,0.00164700,0.02384000,0.21510000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,18.74000000,0.24370000,0.21490000,0.01941000,6.52200000,1.60600000,0.25410000,0.00164600,0.02398000,0.21930000


In [224]:
for item in col[30:45]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,ZECUSDT,IOSTUSDT,CELRUSDT,DASHUSDT,OMGUSDT,THETAUSDT,ENJUSDT,MATICUSDT,ATOMUSDT,TFUELUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,28.07000000,0.00481700,0.00362000,41.84000000,0.62140000,0.08656000,0.07963000,0.01496000,4.38000000,0.00238300
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,27.02000000,0.00460500,0.00340000,40.04000000,0.59500000,0.08443000,0.07371000,0.01467000,4.09600000,0.00229700
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,28.41000000,0.00481900,0.00355000,43.92000000,0.62480000,0.08771000,0.07800000,0.01512000,4.23700000,0.00240300
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,29.91000000,0.00486900,0.00355000,46.19000000,0.62460000,0.09394000,0.07720000,0.01484000,4.25000000,0.00242900
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,30.81000000,0.00491600,0.00349000,51.17000000,0.63670000,0.09421000,0.07637000,0.01483000,4.20000000,0.00241200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,35.90000000,0.01018000,0.02141000,45.43000000,0.98700000,0.96100000,0.36900000,0.97530000,10.84100000,0.04760000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,35.30000000,0.01001000,0.02096000,45.47000000,0.99900000,0.94500000,0.36440000,0.96360000,10.97400000,0.04720000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,33.20000000,0.00931000,0.01880000,41.73000000,0.86000000,0.89300000,0.34310000,0.90050000,10.59100000,0.04390000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,33.30000000,0.00943000,0.01888000,42.03000000,0.88300000,0.89800000,0.34320000,0.87630000,10.37500000,0.04420000


In [225]:
for item in col[45:60]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,FTMUSDT,ALGOUSDT,DOGEUSDT,DUSKUSDT,ANKRUSDT,WINUSDT,COSUSDT,MTLUSDT,TOMOUSDT,PERLUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,0.01058000,0.22010000,0.00202450,0.04260000,0.00143900,0.00009300,0.00948000,0.22770000,0.44580000,0.02064000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,0.01025000,0.21390000,0.00199510,0.03850000,0.00139700,0.00008850,0.00883000,0.21560000,0.41950000,0.02045000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,0.01035000,0.22720000,0.00202360,0.03900000,0.00140700,0.00009110,0.00930000,0.22260000,0.42860000,0.02214000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,0.01068000,0.23520000,0.00205790,0.03910000,0.00141700,0.00009110,0.00930000,0.23270000,0.46950000,0.02238000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,0.01051000,0.22940000,0.00206960,0.03960000,0.00141000,0.00009230,0.00909000,0.22880000,0.44300000,0.02262000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,0.41090000,0.17520000,0.07672000,0.15750000,0.02889000,0.00008040,0.00673000,1.21000000,0.90150000,0.02529000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,0.40440000,0.17320000,0.07652000,0.15650000,0.02858000,0.00008110,0.00639000,1.16700000,0.93560000,0.02472000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,0.36340000,0.16410000,0.07239000,0.14040000,0.02612000,0.00007800,0.00587000,1.04500000,0.91070000,0.02320000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,0.36920000,0.16490000,0.07310000,0.14060000,0.02631000,0.00007760,0.00594000,1.04900000,0.86020000,0.02352000


In [226]:
for item in col[60:75]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,DENTUSDT,DOCKUSDT,WANUSDT,FUNUSDT,CHZUSDT,BANDUSDT,BUSDUSDT,XTZUSDT,RENUSDT,RVNUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,0.00015790,0.00775500,0.17580000,0.00332700,0.00692300,0.23700000,1.00330000,1.37000000,0.03200000,0.02270000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,0.00015860,0.00707100,0.17320000,0.00329300,0.00671900,0.21890000,1.00280000,1.24280000,0.03476000,0.02299000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,0.00016470,0.00733000,0.17950000,0.00334800,0.00715100,0.23180000,1.00110000,1.27850000,0.03556000,0.02300000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,0.00016680,0.00729900,0.17730000,0.00320900,0.00768000,0.23080000,1.00120000,1.25570000,0.03528000,0.02291000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,0.00016390,0.00723500,0.17590000,0.00316900,0.00786900,0.23210000,1.00110000,1.25450000,0.03527000,0.02326000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,0.00091000,0.01828000,0.23370000,0.00543100,0.12510000,1.65300000,0.99930000,0.98000000,0.08411700,0.02218000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,0.00088300,0.01801000,0.22890000,0.00549200,0.12510000,1.62300000,0.99940000,0.96400000,0.08429800,0.02202000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,0.00082700,0.01633000,0.21200000,0.00531100,0.11320000,1.44400000,0.99950000,0.91100000,0.07502700,0.02077000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,0.00082600,0.01640000,0.21420000,0.00531500,0.11270000,1.45500000,0.99990000,0.90600000,0.07744500,0.02112000


In [227]:
for item in col[75:90]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,NKNUSDT,STXUSDT,KAVAUSDT,ARPAUSDT,IOTXUSDT,RLCUSDT,CTXCUSDT,BCHUSDT,TROYUSDT,VITEUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,0.01809000,0.09890000,1.13640000,0.01064000,0.00345700,0.41770000,0.06400000,204.44000000,0.00677870,0.01104000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,0.01845000,0.08970000,1.01720000,0.00985000,0.00335800,0.37640000,0.06140000,195.49000000,0.00647510,0.01042000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,0.01820000,0.09450000,1.02410000,0.01007000,0.00341000,0.40410000,0.06330000,222.43000000,0.00643210,0.01123000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,0.01785000,0.09440000,1.04010000,0.01022000,0.00354300,0.39810000,0.06980000,224.47000000,0.00640100,0.01170000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,0.01868000,0.09200000,1.03100000,0.01008000,0.00356500,0.40500000,0.06680000,222.79000000,0.00656840,0.01109000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,0.10650000,0.70000000,0.72400000,0.03957000,0.02357000,1.80500000,0.18120000,117.00000000,0.00293300,0.01993000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,0.10560000,0.73140000,0.72600000,0.04270000,0.02331000,1.76000000,0.18020000,116.10000000,0.00289000,0.01976000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,0.09220000,0.74100000,0.74500000,0.03921000,0.02226000,1.67500000,0.16460000,111.10000000,0.00262800,0.01716000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,0.09530000,0.69180000,0.80200000,0.03930000,0.02229000,1.53900000,0.16730000,122.20000000,0.00262700,0.01745000


In [228]:
for item in col[90:115]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,NKNUSDT,STXUSDT,KAVAUSDT,ARPAUSDT,IOTXUSDT,RLCUSDT,CTXCUSDT,BCHUSDT,TROYUSDT,VITEUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,0.01809000,0.09890000,1.13640000,0.01064000,0.00345700,0.41770000,0.06400000,204.44000000,0.00677870,0.01104000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,0.01845000,0.08970000,1.01720000,0.00985000,0.00335800,0.37640000,0.06140000,195.49000000,0.00647510,0.01042000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,0.01820000,0.09450000,1.02410000,0.01007000,0.00341000,0.40410000,0.06330000,222.43000000,0.00643210,0.01123000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,0.01785000,0.09440000,1.04010000,0.01022000,0.00354300,0.39810000,0.06980000,224.47000000,0.00640100,0.01170000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,0.01868000,0.09200000,1.03100000,0.01008000,0.00356500,0.40500000,0.06680000,222.79000000,0.00656840,0.01109000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,0.10650000,0.70000000,0.72400000,0.03957000,0.02357000,1.80500000,0.18120000,117.00000000,0.00293300,0.01993000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,0.10560000,0.73140000,0.72600000,0.04270000,0.02331000,1.76000000,0.18020000,116.10000000,0.00289000,0.01976000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,0.09220000,0.74100000,0.74500000,0.03921000,0.02226000,1.67500000,0.16460000,111.10000000,0.00262800,0.01716000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,0.09530000,0.69180000,0.80200000,0.03930000,0.02229000,1.53900000,0.16730000,122.20000000,0.00262700,0.01745000


In [229]:
for item in col[115:130]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,NKNUSDT,STXUSDT,KAVAUSDT,ARPAUSDT,IOTXUSDT,RLCUSDT,CTXCUSDT,BCHUSDT,TROYUSDT,VITEUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,0.01809000,0.09890000,1.13640000,0.01064000,0.00345700,0.41770000,0.06400000,204.44000000,0.00677870,0.01104000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,0.01845000,0.08970000,1.01720000,0.00985000,0.00335800,0.37640000,0.06140000,195.49000000,0.00647510,0.01042000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,0.01820000,0.09450000,1.02410000,0.01007000,0.00341000,0.40410000,0.06330000,222.43000000,0.00643210,0.01123000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,0.01785000,0.09440000,1.04010000,0.01022000,0.00354300,0.39810000,0.06980000,224.47000000,0.00640100,0.01170000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,0.01868000,0.09200000,1.03100000,0.01008000,0.00356500,0.40500000,0.06680000,222.79000000,0.00656840,0.01109000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,0.10650000,0.70000000,0.72400000,0.03957000,0.02357000,1.80500000,0.18120000,117.00000000,0.00293300,0.01993000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,0.10560000,0.73140000,0.72600000,0.04270000,0.02331000,1.76000000,0.18020000,116.10000000,0.00289000,0.01976000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,0.09220000,0.74100000,0.74500000,0.03921000,0.02226000,1.67500000,0.16460000,111.10000000,0.00262800,0.01716000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,0.09530000,0.69180000,0.80200000,0.03930000,0.02229000,1.53900000,0.16730000,122.20000000,0.00262700,0.01745000


In [230]:
for item in col[130:151]:
    a = price(item)
    if a.empty == False:
        if len(a) == 1226:
            df_price[item] = a
    time.sleep(3)

df_price

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,IOTAUSDT,...,NKNUSDT,STXUSDT,KAVAUSDT,ARPAUSDT,IOTXUSDT,RLCUSDT,CTXCUSDT,BCHUSDT,TROYUSDT,VITEUSDT
0,7200.85000000,130.77000000,13.71840000,8.85000000,41.62000000,1.60300000,0.03348000,0.19305000,2.60390000,0.16060000,...,0.01809000,0.09890000,1.13640000,0.01064000,0.00345700,0.41770000,0.06400000,204.44000000,0.00677870,0.01104000
1,6965.71000000,127.19000000,13.01050000,8.46200000,39.43000000,1.53300000,0.03269000,0.18750000,2.45930000,0.15840000,...,0.01845000,0.08970000,1.01720000,0.00985000,0.00335800,0.37640000,0.06140000,195.49000000,0.00647510,0.01042000
2,7344.96000000,134.35000000,13.64740000,8.88800000,42.23000000,1.62900000,0.03417000,0.19338000,2.63960000,0.16640000,...,0.01820000,0.09450000,1.02410000,0.01007000,0.00341000,0.40410000,0.06330000,222.43000000,0.00643210,0.01123000
3,7354.11000000,134.20000000,13.80920000,9.07700000,42.78000000,1.64200000,0.03434000,0.19305000,2.64810000,0.17340000,...,0.01785000,0.09440000,1.04010000,0.01022000,0.00354300,0.39810000,0.06980000,224.47000000,0.00640100,0.01170000
4,7358.75000000,135.37000000,14.05320000,9.02000000,43.30000000,1.63100000,0.03460000,0.19453000,2.68650000,0.17180000,...,0.01868000,0.09200000,1.03100000,0.01008000,0.00356500,0.40500000,0.06680000,222.79000000,0.00656840,0.01109000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,28848.20000000,1896.33000000,321.90000000,10.01000000,83.26000000,2.77700000,0.37870000,0.45820000,0.98300000,0.19210000,...,0.10650000,0.70000000,0.72400000,0.03957000,0.02357000,1.80500000,0.18120000,117.00000000,0.00293300,0.01993000
1222,28430.10000000,1870.40000000,321.00000000,9.88000000,83.42000000,2.71700000,0.37600000,0.44900000,0.96900000,0.19180000,...,0.10560000,0.73140000,0.72600000,0.04270000,0.02331000,1.76000000,0.18020000,116.10000000,0.00289000,0.01976000
1223,27668.79000000,1847.56000000,313.80000000,9.13000000,77.71000000,2.57500000,0.36530000,0.42830000,0.90600000,0.18030000,...,0.09220000,0.74100000,0.74500000,0.03921000,0.02226000,1.67500000,0.16460000,111.10000000,0.00262800,0.01716000
1224,27628.27000000,1846.52000000,312.10000000,9.26000000,79.90000000,2.61000000,0.36330000,0.42890000,0.91900000,0.17960000,...,0.09530000,0.69180000,0.80200000,0.03930000,0.02229000,1.53900000,0.16730000,122.20000000,0.00262700,0.01745000


Уже начиная с 90-го запроса видно, что далее идут более новые криптовалюты, котрые существуют менее 2 лет (мы их отфильтровываем при отправке запроса, чтобы сравнение за выбранный период было более корректным).

Итого получаем таблицу с котировками 68 криптовалют за 1225 дней с 01.01.2020 до 10.05.2023.

In [231]:
# Экспортируем таблицу в формат csv
df_price.to_csv('df_price.csv')